In [ ]:
import csv
from langchain.docstore.document import Document 
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Define the columns we want to embed vs which ones we want in metadata
columns_to_emebd = ["Description","Features"]
columns_to_metadata = ["Product Name","Price", "Rating","Description", "Features"]